# Project 7 - Create a model to predict the revenues of potential customers using ANOVA and Linear Regression.

🎯 Goal: Create a model to predict the future revenue of children based on their location, the revenue of their parents and the gini index; in order to target future customers.

## Part 3 - Preprocessing

### Get started

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats  
import seaborn as sns
import missingno as msno 
from collections import Counter
import scipy.stats as st

%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [106]:
# Read the data
quant = pd.read_csv(Path.cwd()/'quantile.csv',index_col=0)
inc = pd.read_csv(Path.cwd()/'worldbank.csv',index_col=0)
inc_avg = pd.read_csv(Path.cwd()/'worldbank_avg.csv')

/Users/aureliegiraud/.virtualenvs/env3.8/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


#### Extract conditional probabilities

In [60]:
quant.head()

,name,y_child,y_parent,c_i_child,c_i_parent
0,Albania,4.854682,1.643313,93,70
1,Albania,1.908174,0.870868,72,45
2,Albania,0.294771,1.911118,14,75
3,Albania,2.123639,4.586099,76,94
4,Albania,1.655759,0.791240,68,41


In [64]:
quant.name.nunique()

51

<b> Only 51 countries are covered by conditionnal probabilities: </b> It's due to some missing pj in Part 2.

In [61]:
# Get a dataframe with the conditional probability by combination of classes
proba_cond = quant.groupby(['name','c_i_child','c_i_parent']).count().iloc[:,0].reset_index()
proba_cond['proba_cond'] = proba_cond['y_child']/1000
proba_cond.drop(['y_child'], axis=1,inplace=True)
proba_cond.head()

,name,c_i_child,c_i_parent,proba_cond
0,Albania,1,1,0.120
1,Albania,1,2,0.060
2,Albania,1,3,0.047
3,Albania,1,4,0.039
4,Albania,1,5,0.030


In [62]:
# Check the results: we must have 100% on the sum of conditional probabilities per country
tmp = proba_cond.groupby('name').sum().reset_index()
tmp.head()

,name,c_i_child,c_i_parent,proba_cond
0,Albania,496900,497482,100.0
1,Austria,503480,503661,100.0
2,Belarus,503037,503032,100.0
3,Belgium,503480,503661,100.0
4,Bosnia and Herzegovina,496900,497482,100.0


In [63]:
# Count total individuals to attribute by combination of classes
proba_cond['attribution'] = proba_cond['proba_cond']*500
proba_cond['attribution'] = proba_cond['attribution'].astype(int)
proba_cond.drop(['proba_cond'],axis=1,inplace=True)
proba_cond.head()

,name,c_i_child,c_i_parent,attribution
0,Albania,1,1,60
1,Albania,1,2,30
2,Albania,1,3,23
3,Albania,1,4,19
4,Albania,1,5,15


<b> It means (for example) that we need to attribute the class 1 to 60 rows where child class = 1 and country = Albania, into the World Bank Income dataframe </b>

In [73]:
# Check output
tmp = proba_cond.groupby(['name','c_i_child']).sum()[['attribution']].reset_index()
tmp.head(15)

,name,c_i_child,attribution
0,Albania,1,477
1,Albania,2,473
2,Albania,3,476
3,Albania,4,473
4,Albania,5,474
5,Albania,6,477
6,Albania,7,478
7,Albania,8,476
8,Albania,9,474
9,Albania,10,476


In [74]:
tmp[tmp.attribution >= 500]

,name,c_i_child,attribution


<b> We should have, at maximum, a sum of 500 individuals for each child's class, which is the case here </b> 

In [78]:
# Create a function that duplicates each combination of classes by the corresponding number of parents
def duplicates(data):
    attribut = pd.concat([data]*data['attribution'].values[0], ignore_index=True)
    attribut = attribut[['attribution']]
    return attribut

In [79]:
# Remove the rows with zero values
tmp = proba_cond[proba_cond['attribution'] > 0]

In [111]:
# Create a table with the number of individuals by combinations based on the conditional probabilities 
ref = tmp.groupby(['name','c_i_child','c_i_parent']).apply(duplicates)
ref.head()

attribution
name    c_i_child c_i_parent               
Albania 1         1          0           60
                             1           60
                             2           60
                             3           60
                             4           60

#### Creation of Final Dataset for Analysis

In [114]:
inc['quantile'] = inc['quantile'].astype(int)
inc.head()

,year_survey,quantile,nb_quantiles,income,gdpppp,name
2,2008.0,1,100.0,728.89795,7297.0,Albania
3,2008.0,2,100.0,916.66235,7297.0,Albania
4,2008.0,3,100.0,1010.91600,7297.0,Albania
5,2008.0,4,100.0,1086.90780,7297.0,Albania
6,2008.0,5,100.0,1132.69970,7297.0,Albania


In [115]:
# Merge with the World Bank Income dataset
final = pd.merge(ref.reset_index(), inc_avg[['name','new_gini','income']], on = 'name', how = 'inner')
final.rename(columns= {'income':'avg_income'},inplace=True)
final = pd.merge(final,inc[['name','quantile','income']], left_on=['name','c_i_child'],right_on=['name','quantile'], how = 'left')
final.drop(['level_3','c_i_child','attribution','quantile'],axis=1,inplace=True)
final.head()

,name,c_i_parent,new_gini,avg_income,income
0,Albania,1,32.141019,2994.829902,728.89795
1,Albania,1,32.141019,2994.829902,728.89795
2,Albania,1,32.141019,2994.829902,728.89795
3,Albania,1,32.141019,2994.829902,728.89795
4,Albania,1,32.141019,2994.829902,728.89795


In [116]:
len(final)

2424946

In [117]:
final.to_csv(Path.cwd()/'final_dataset.csv')